<a href="https://colab.research.google.com/github/christianlanx/mynewrepository/blob/master/ECoS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains a program which extracts the contents of the ECoS Manuscript DB Google Sheet (our current comment database), as a csv file, and writes it out to a json file for use in the ECoS App

This is done by stepping through the spreadsheet csv file in a single pass for each loop
  * each line of the spreadsheet is a python dictionary
  * if a line is the start of a new chapter, we write the chapter header
  * if a line is the start of a new section, we write the section header
  * if a line is the start of a new subsection, we write the subsection header
  * every line is an item, so we always write the item
  * **Before we write a new heading, we close out the old one at the same level, you'll see it in the code**



*Another way we could go about doing it is to put everything into one big dictionary that has the same structure as the json file, that way it would be easier to write a bunch of nested for loops to go through that dictionary instead, but doing it this way works fine for now...*

**STRUCTURE OF THE ECoS DOCUMENT**:

1.A.1.A is the reference number of a particular comment
 1 . is the chapter number
 1.A. is section A of chapter 1
 1.A.1. is subsection 1 of section A of chapter 1
 1.A.1.A is the first item in that particular subsection

Note: there a few items out there that don't really conform to this standard, they have weird numbers like (1.A.1.A.a.1) and since the app doesn't really have any way to deal them as of right now, we will simply have to skip over them. I would like to maybe have a record of them to look at though, so we can see if we missed anything.

I would suggest you take a look at the json file before you work on this to get an idea of what the format looks like. Look for Manuscript.js in the github.
This script doesn't write the whole thing (but it totally can), just the main block which has all the items in it.
After you run this script, the output gets manually pasted into the Manuscript.js file.
Another thing we can do is just write to the whole file and overwrite the existing file...

In [0]:
import csv

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# run this cell to print out the csv file, for reference
path_to_file = './drive/My Drive/ECoS Developer/ECoS Website/ECoS Manuscript DB - Comments.csv'
csvfile = open(path_to_file, newline='')
reader = csv.DictReader(csvfile)
for row in reader:
  print(row)
# click the thing on the top left of the output to close it

FileNotFoundError: ignored

In [0]:
# The following methods take the content of the ecos doc spreadsheet, and print
# it out to the json format

# write the chapter header
# idk what this stuff is, but it goes at the start of each chapter
def writeChapterHeader(content, chapter):
  content.append("{\n");
  content.append("name: \'" + chapter + "\',\n")
  content.append("expandAriaLabel: \'Expand Basic components section\',\n")
  content.append("collapseAriaLabel: \'Collapse Basic components section', \n")
  content.append("collapseByDefault: true,\n")
  content.append("links: [")

# write the chapter footer, to close out the chapter
# after the last chapter we close up this whole block
def writeChapterFooter(content):
  content.append("]\n")
  content.append("},")

#section header
def writeSectionHeader(content, section):
  content.append("{\n")
  content.append("key: \'" + section + "\',\n")
  content.append("name: \'" + section + "\',\n")
  content.append("link: \'/ecos\',\n")

#section footer
def writeSectionFooter(content):
  content.append("},\n")
  #content.append()
## TODO: you shouldn't need to mess with any of the above methods, just focus
## on the subsection header

## you need to make it so that the subsection header and footer always get
## written out, even if there aren't any efficiency, content or style items
### Do not make subsections for anything but efficiency content and style!

# subsection header
# efficiencyItems: []
def writeSubSectHeader(content, subsect):
  content.append(subsect.lower() + "Items : [")

# i dont put a comma after these, because the json file doesn't like extra
# commas at the end of the list of subsections
      ## extra commas are totally fine at the ends the other lists here
      ## if you look at the other footer methods
def writeSubSectFooter(content):
  content.append("]")

# this method writes out the item, and it puts a comment at the end of the line
# with some extra info that isn't being currently used
def writeItem(content, title, link, refnum, status):
  content.append("{key:\"" + title + "\", link: \"" + link + "\"},")
  content.append("\t// id: \"" + refnum + "\", status: \"" + status + "\"\n")

In [0]:
# open the csv file, note that this file is in my google drive
# I saved the google sheet as csv file and put it in my ECoS Developer gdrive
path_to_file = './drive/My Drive/ECoS Developer/ECoS Website/ECoS Manuscript DB - Comments.csv'
csvfile = open(path_to_file, newline='')
# open a new DictReader object, which converts each line to a dictionary
reader = csv.DictReader(csvfile)
# gets the next DictReader, since its an iterable
row = next(reader)
# get the field 'Number'
refnum = row['Number']
# get the title, and replace any single or double quotes with \" and \'...
# we have to do this for any text really, since it's going into a json format
# 1.A.1.A Headings should be consistently capitalized.
# title is the comment itself, or the title of it
title = row['Title'].replace("\'", "\\\'").replace("\"", "\\\"")
# 1. Building Blocks, 2. Sections of Manuscript... --- chapter
chapter = row['Chapter'].replace("\'", "\\\'").replace("\"", "\\\"")
# 1.A. Headings, 1.B. Figures --- section
section = row['Section'].replace("\'", "\\\'").replace("\"", "\\\"")
# 1.A.1. Efficiency --- subsection
subsect = row['Subsection'].replace("\'", "\\\'").replace("\"", "\\\"")
# Ready for Review, Started, Todo ==== Status
status = row['Status']
# The link to the spot on the ECoS website for this particular comment
link = row['LINK']

# set the current chapter, current section, current subsection
chapter_curr = chapter
section_curr = section
subsect_curr = subsect

# So content is a list, which will be converted to a string at the end of this
# program. It's basically the content that will be pasted into the json file
content = []
# first line of content
content.append("groups={[\n")
# write the first chapter header, section header, subsection and comment item
writeChapterHeader(content, chapter)
writeSectionHeader(content, section)
writeSubSectHeader(content, subsect)
writeItem(content, title, link, refnum, status)

# this is the main loop of the program, which iterates over the remaining rows
# in the csv Dict Reader object
for row in reader:

  # get all the data from the row, making sure to escape any quotes
  refnum = row['Number']
  title = row['Title'].replace("\'", "\\\'").replace("\"", "\\\"")
  chapter = row['Chapter'].replace("\'", "\\\'").replace("\"", "\\\"")
  section = row['Section'].replace("\'", "\\\'").replace("\"", "\\\"")
  subsect = row['Subsection'].replace("\'", "\\\'").replace("\"", "\\\"")
  status = row['Status']
  link = row['LINK']


  ## TODO: This is where you need to redo the logic of the program,
  ## Make it so that every Section has efficiency items, content items, and
  ## style items...
  ## you will have to rewrite these next lines of code, so take your time and
  ## let me know when you're ready to get started
  
  # this statement says that for any item where the subsection is not
  # Efficiency, content, or Style, we will just skip over it and move on to the
  # next one
  if (subsect != 'Efficiency' and subsect != 'Content' and subsect != 'Style'):
    continue

  # if this line is the start of a new subsection, update the current subsection
  # and close out the old one
  if (subsect != subsect_curr):
    subsect_curr = subsect
    writeSubSectFooter(content)

    # if this line is the start of a new section, update the current section
    # add a newline for formatting (we don't really care about formatting)
    # and write the section footer to close out the previous section
    if (section != section_curr):
      content.append('\n')
      section_curr = section
      writeSectionFooter(content)

      # if this line is a new chapter, update the current chapter
      # write a chapter footer
      # then write the new chapter header
      if (chapter != chapter_curr):
        chapter_curr = chapter
        writeChapterFooter(content)
        writeChapterHeader(content, chapter)
        # end of new chapter logic

      # write the section header after the new chapter header, if there is one
      writeSectionHeader(content, section)
      # end of new section logic

    # if it's not a new section, put a comma and a newline
    # for some reason, you cant have an extra comma after the last subsection in
    # the list of sections, which is why I have this else statement
    else:
      # SO, THESE COMMAS GO IN BETWEEN THE SUBSECTIONS!!
      content.append(',\n');
      # end of section logic

    # write the subsection header, after writing the other headers, if aplicable
    writeSubSectHeader(content, subsect)

  #finally, always writeItem, since each line is an item
  writeItem(content, title, link, refnum, status)

  #end of for each loop

## close up the document
writeSubSectFooter(content)
writeSectionFooter(content)
writeChapterFooter(content)
content.append("\n")
content.append("]}")

## turn content into one big string and print it out
## could also write it out to a file...
c = ''.join([x for x in content])
print(c)

groups={[
{
name: 'Building Blocks',
expandAriaLabel: 'Expand Basic components section',
collapseAriaLabel: 'Collapse Basic components section', 
collapseByDefault: true,
links: [{
key: 'Headings',
name: 'Headings',
link: '/ecos',
efficiencyItems : [{key:"Important words in a heading should be capitalized consistently.", link: "https://sites.google.com/uw.edu/ecostechnicaldocumentation/1-building-blocks/1-a-headings#h.km23sldq0i7k"},	// id: "1.A.1.A", status: "Ready for review"
{key:"All headings should be numbered automatically.", link: "https://sites.google.com/uw.edu/ecostechnicaldocumentation/1-building-blocks/1-a-headings#h.1jbwqsvodhov"},	// id: "1.A.1.B", status: "Ready for review"
{key:"Check heading numbering and position on the page to avoid orphan headings.", link: "https://sites.google.com/uw.edu/ecostechnicaldocumentation/1-building-blocks/1-a-headings#h.9l0olz6z30"},	// id: "1.A.1.C", status: "Ready for review"
{key:"Cross-reference headings that are mentioned inside the 